In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

2024-03-29 12:52:56.329538: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 12:52:56.329650: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 12:52:56.441315: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os
import pandas as pd
from PIL import Image
import numpy as np
from tqdm import tqdm
import cv2

In [3]:
train_df = pd.read_csv('/kaggle/input/vista-codefest-1/dataset/train.csv')
bbox_df = pd.read_csv('/kaggle/input/vista-codefest-1/dataset/bbox.csv')

In [6]:
def preprocess_images(image_folder, output_folder, df, size=(224, 224)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, row in tqdm(df.iterrows(), total=len(df)):
        image_path = os.path.join(image_folder, row['Name'])
        image = Image.open(image_path)
        image = image.resize(size)
        image = np.array(image)

        # Check if the image has three dimensions (color channels)
        if len(image.shape) == 3:
            # Apply histogram equalization
            for channel in range(image.shape[2]):
                image[:, :, channel] = cv2.equalizeHist(image[:, :, channel])

            # Apply Gaussian blurring
            image = cv2.GaussianBlur(image, (5, 5), 0)

        image = image / 255.0  # Normalize pixel values
        image = (image * 255).astype(np.uint8)  # Convert back to 8-bit values

        output_path = os.path.join(output_folder, row['Name'])
        cv2.imwrite(output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))



# Preprocess training images
preprocess_images('/kaggle/input/vista-codefest-1/dataset/train', '/kaggle/working/dataset/preprocessed/train', train_df)

# Preprocess testing images (assuming you have a test.csv similar to train.csv)
test_df = pd.read_csv('/kaggle/input/vista-codefest-1/dataset/test.csv')
preprocess_images('/kaggle/input/vista-codefest-1/dataset/test', '/kaggle/working/dataset/preprocessed/test', test_df)



100%|██████████| 3963/3963 [01:06<00:00, 59.43it/s]


In [7]:
def process_bounding_boxes(df, image_folder):
    processed_boxes = {}
    for _, row in tqdm(df.iterrows(), total=len(df)):
        image_name = row['Name']
        if image_name not in processed_boxes:
            processed_boxes[image_name] = []
        processed_boxes[image_name].append([row['xmin'], row['ymin'], row['xmax'], row['ymax']])

    # Save or use the processed_boxes dictionary as needed
    return processed_boxes

bbox_dict = process_bounding_boxes(bbox_df, '/kaggle/input/vista-codefest-1/dataset/train')


100%|██████████| 62529/62529 [00:04<00:00, 13201.20it/s]


In [27]:
# Load the preprocessed data
train_df = pd.read_csv('/kaggle/input/vista-codefest-1/dataset/train.csv')
train_df['HeadCount'] = train_df['HeadCount'].astype('float32')
test_df = pd.read_csv('/kaggle/input/vista-codefest-1/dataset/test.csv')
test_df['HeadCount'] = test_df['HeadCount'].astype('float32')

# Split the data into training and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)


# Set up image data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/kaggle/working/dataset/preprocessed/train',
    x_col='Name',
    y_col='HeadCount',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_data,
    directory='/kaggle/working/dataset/preprocessed/train',
    x_col='Name',
    y_col='HeadCount',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

Found 8186 validated image filenames.
Found 2047 validated image filenames.


In [29]:
from tensorflow.keras.applications import EfficientNetB3

# Define the create_model function
def create_model(base_model):
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    return model

# EfficientNetB3 model
efficientnet_base = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
efficientnet_model = create_model(efficientnet_base)
efficientnet_model.fit(train_generator, epochs=15, validation_data=val_generator)

Epoch 1/15


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


256/256 ━━━━━━━━━━━━━━━━━━━━ 395s 904ms/step - loss: 16.0921 - rmse: 3.9705 - val_loss: 28.8995 - val_rmse: 5.3757
Epoch 2/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 107s 405ms/step - loss: 9.3873 - rmse: 3.0614 - val_loss: 24.6603 - val_rmse: 4.9661
Epoch 3/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 107s 407ms/step - loss: 9.0444 - rmse: 3.0064 - val_loss: 7.5891 - val_rmse: 2.7549
Epoch 4/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 107s 406ms/step - loss: 7.6963 - rmse: 2.7735 - val_loss: 7.1093 - val_rmse: 2.6663
Epoch 5/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 106s 403ms/step - loss: 7.2534 - rmse: 2.6919 - val_loss: 6.8966 - val_rmse: 2.6261
Epoch 6/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 107s 405ms/step - loss: 6.8675 - rmse: 2.6199 - val_loss: 6.1652 - val_rmse: 2.4828
Epoch 7/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 106s 404ms/step - loss: 6.5522 - rmse: 2.5593 - val_loss: 5.9470 - val_rmse: 2.4385
Epoch 8/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 109s 414ms/step - loss: 6.2176 - rmse: 2.4902 - val_loss: 6.1148 - val_rmse: 2.4723
Epoch 9/15
256/2

In [30]:
# Assume 'test_generator' is already created similar to 'train_generator' but without shuffling
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='/kaggle/working/dataset/preprocessed/test',
    x_col='Name',
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

# Make predictions on the test set
predictions = efficientnet_model.predict(test_generator)

# Prepare submission DataFrame
submission_df = pd.DataFrame({
    'Name': test_df['Name'],
    'HeadCount': predictions.flatten()
})

# Save to CSV
submission_df.to_csv('/kaggle/working/submission_efficientnet_optimized.csv', index=False)


Found 3963 validated image filenames.
124/124 ━━━━━━━━━━━━━━━━━━━━ 26s 144ms/step


In [31]:
def create_model_finetuned(base_model, unfreeze_layers):
    # Unfreeze the top layers
    base_model.trainable = True
    for layer in base_model.layers[:-unfreeze_layers]:
        layer.trainable = False

    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    return model

# Create and train a fine-tuned model
efficientnet_model_finetuned = create_model_finetuned(efficientnet_base, unfreeze_layers=20)
efficientnet_model_finetuned.fit(train_generator, epochs=15, validation_data=val_generator)

Epoch 1/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 214s 599ms/step - loss: 11.4016 - rmse: 3.2934 - val_loss: 5.6454 - val_rmse: 2.3759
Epoch 2/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 395ms/step - loss: 4.4581 - rmse: 2.1104 - val_loss: 5.4770 - val_rmse: 2.3402
Epoch 3/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 106s 401ms/step - loss: 5.2144 - rmse: 2.2760 - val_loss: 5.3753 - val_rmse: 2.3188
Epoch 4/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 398ms/step - loss: 3.9976 - rmse: 1.9981 - val_loss: 5.5804 - val_rmse: 2.3625
Epoch 5/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 396ms/step - loss: 4.5106 - rmse: 2.1211 - val_loss: 5.5653 - val_rmse: 2.3584
Epoch 6/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 106s 403ms/step - loss: 4.8272 - rmse: 2.1909 - val_loss: 5.3381 - val_rmse: 2.3103
Epoch 7/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 396ms/step - loss: 4.1630 - rmse: 2.0394 - val_loss: 5.2719 - val_rmse: 2.2959
Epoch 8/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 395ms/step - loss: 4.4449 - rmse: 2.1054 - val_loss: 5.2328 - val_rmse: 2.2876
Epoch 9

In [34]:
# Assume 'test_generator' is already created similar to 'train_generator' but without shuffling
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='/kaggle/working/dataset/preprocessed/test',
    x_col='Name',
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

# Make predictions on the test set
predictions = efficientnet_model_finetuned.predict(test_generator)

# Prepare submission DataFrame
submission_df = pd.DataFrame({
    'Name': test_df['Name'],
    'HeadCount': predictions.flatten()
})

# Save to CSV
submission_df.to_csv('/kaggle/working/submission_efficientnet_finetuned_optimized.csv', index=False)

Found 3963 validated image filenames.
124/124 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step


In [35]:
def create_custom_model(base_model):
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    return model

# Create and train a custom model
custom_model = create_custom_model(efficientnet_base)
custom_model.fit(train_generator, epochs=15, validation_data=val_generator)

Epoch 1/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 216s 607ms/step - loss: 9.0494 - rmse: 2.9449 - val_loss: 6.1436 - val_rmse: 2.4781
Epoch 2/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 395ms/step - loss: 5.1918 - rmse: 2.2751 - val_loss: 6.3937 - val_rmse: 2.5287
Epoch 3/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 393ms/step - loss: 5.1926 - rmse: 2.2784 - val_loss: 8.7455 - val_rmse: 2.9575
Epoch 4/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 397ms/step - loss: 4.7991 - rmse: 2.1900 - val_loss: 12.8431 - val_rmse: 3.5839
Epoch 5/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 106s 402ms/step - loss: 4.8138 - rmse: 2.1932 - val_loss: 8.9895 - val_rmse: 2.9986
Epoch 6/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 394ms/step - loss: 4.2891 - rmse: 2.0696 - val_loss: 8.5042 - val_rmse: 2.9161
Epoch 7/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 103s 393ms/step - loss: 4.5805 - rmse: 2.1395 - val_loss: 10.6101 - val_rmse: 3.2573
Epoch 8/15
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 395ms/step - loss: 4.3474 - rmse: 2.0809 - val_loss: 11.4575 - val_rmse: 3.3847
Epoch

In [36]:
# Make predictions with the EfficientNet model
efficientnet_predictions = efficientnet_model.predict(test_generator)

# Ensemble predictions from VGG16, ResNet50, and EfficientNet
#ensemble_predictions = (vgg_predictions.flatten() + resnet_predictions.flatten() + efficientnet_predictions.flatten()) / 3
ensemble_predictions = efficientnet_predictions.flatten()

# Round the ensemble predictions to the nearest integer
ensemble_head_counts = ensemble_predictions

# Create a submission DataFrame for the ensemble
ensemble_submission_df = pd.DataFrame({
    'Name': test_df['Name'],
    'HeadCount': ensemble_head_counts
})

# Save the ensemble submission DataFrame to a CSV file
ensemble_submission_df.to_csv('/kaggle/working/ensemble_submission.csv', index=False)

124/124 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step


In [37]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import pandas as pd


In [42]:
train_df = pd.read_csv('/kaggle/input/vista-codefest-1/dataset/train.csv')
train_df['HeadCount'] = train_df['HeadCount'].astype('float32')
test_df = pd.read_csv('/kaggle/input/vista-codefest-1/dataset/test.csv')
test_df['HeadCount'] = test_df['HeadCount'].astype('float32')

# Split the data into training and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Set up image data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/kaggle/working/dataset/preprocessed/train',
    x_col='Name',
    y_col='HeadCount',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_data,
    directory='/kaggle/working/dataset/preprocessed/train',
    x_col='Name',
    y_col='HeadCount',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)


Found 8186 validated image filenames.
Found 2047 validated image filenames.


In [43]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Load a pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Create a sequential model and add the base model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='linear'))  # Output layer for regression

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])


In [44]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-03-29 16:47:38.471454: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 4.63498, expected 3.85968
2024-03-29 16:47:38.471506: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 6.55784, expected 5.78254
2024-03-29 16:47:38.471515: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 6.61282, expected 5.83752
2024-03-29 16:47:38.471523: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 6.34499, expected 5.56968
2024-03-29 16:47:38.471531: E external/local_xla/xla/service/gpu/buffer_comparator.cc:11

213/256 ━━━━━━━━━━━━━━━━━━━━ 16s 389ms/step - loss: 15.2929 - rmse: 3.8814

2024-03-29 16:49:17.432254: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.46443, expected 2.59124
2024-03-29 16:49:17.432307: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 4.84232, expected 3.96913
2024-03-29 16:49:17.432316: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 4.91526, expected 4.04207
2024-03-29 16:49:17.432324: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 5.02814, expected 4.15494
2024-03-29 16:49:17.432331: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 4.53562, expected 3.66243
2024-03-29 16:49:17.432339: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 4.79722, expected 3.92403
2024-03-29 16:49:17.432358: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 4.69349, expected 3.82029
2024-03-29 16:49:17.432366: E external/local_xla/xla/se

256/256 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step - loss: 14.8698 - rmse: 3.8299

2024-03-29 16:49:48.564050: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 100352: 3.62212, expected 2.70127
2024-03-29 16:49:48.564100: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 100353: 5.31563, expected 4.39477
2024-03-29 16:49:48.564109: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 100354: 5.43093, expected 4.51007
2024-03-29 16:49:48.564117: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 100355: 5.35723, expected 4.43638
2024-03-29 16:49:48.564124: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 100356: 5.06878, expected 4.14793
2024-03-29 16:49:48.564132: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 100357: 5.791, expected 4.87014
2024-03-29 16:49:48.564140: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 100358: 5.23388, expected 4.31303
2024-03-29 16:49:48.56

256/256 ━━━━━━━━━━━━━━━━━━━━ 149s 501ms/step - loss: 14.8610 - rmse: 3.8288 - val_loss: 9.3074 - val_rmse: 3.0502
Epoch 2/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 106s 402ms/step - loss: 10.2421 - rmse: 3.1995 - val_loss: 10.0781 - val_rmse: 3.1746
Epoch 3/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 103s 393ms/step - loss: 9.5269 - rmse: 3.0832 - val_loss: 8.3703 - val_rmse: 2.8932
Epoch 4/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 106s 404ms/step - loss: 9.7840 - rmse: 3.1270 - val_loss: 9.4452 - val_rmse: 3.0732
Epoch 5/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 396ms/step - loss: 9.1802 - rmse: 3.0297 - val_loss: 8.3524 - val_rmse: 2.8900
Epoch 6/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 395ms/step - loss: 9.4944 - rmse: 3.0803 - val_loss: 7.4779 - val_rmse: 2.7349
Epoch 7/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 395ms/step - loss: 9.3007 - rmse: 3.0468 - val_loss: 7.9487 - val_rmse: 2.8191
Epoch 8/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 104s 395ms/step - loss: 8.7159 - rmse: 2.9492 - val_loss: 7.7942 - val_rmse: 2.7917
Epoch 9/10
256/2

In [46]:
test_df = pd.read_csv('/kaggle/input/vista-codefest-1/dataset/test.csv')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='/kaggle/working/dataset/preprocessed/test',
    x_col='Name',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False  # Important for maintaining order
)

predictions = model.predict(test_generator)


# Round the predictions to the nearest integer
predicted_head_counts = predictions.flatten()

# Create a DataFrame for the submission
submission_df = pd.DataFrame({
    'Name': test_df['Name'],
    'HeadCount': predicted_head_counts
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('/kaggle/working/submission_main_vgg_kaggle.csv', index=False)


Found 3963 validated image filenames.


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


123/124 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step

2024-03-29 17:06:01.719372: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.99165, expected 3.3217
2024-03-29 17:06:01.719429: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 7: 5.28926, expected 4.61931
2024-03-29 17:06:01.719439: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 8: 5.06719, expected 4.39724
2024-03-29 17:06:01.719447: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 11: 5.46955, expected 4.7996
2024-03-29 17:06:01.719454: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 12: 3.74731, expected 3.07735
2024-03-29 17:06:01.719462: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 13: 3.78108, expected 3.11113
2024-03-29 17:06:01.719469: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 14: 5.36791, expected 4.69796
2024-03-29 17:06:01.719477: E external/local_xla/xla/

124/124 ━━━━━━━━━━━━━━━━━━━━ 20s 160ms/step


In [47]:
#advanced model

In [48]:
import numpy as np
import pandas as pd
import tensorflow as tf  # Add this line to import TensorFlow
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [51]:
train_df = pd.read_csv('/kaggle/input/vista-codefest-1/dataset/train.csv')
train_df['HeadCount'] = train_df['HeadCount'].astype('float32')
test_df = pd.read_csv('/kaggle/input/vista-codefest-1/dataset/test.csv')
test_df['HeadCount'] = test_df['HeadCount'].astype('float32')

# Split the data into training and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Set up image data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/kaggle/working/dataset/preprocessed/train',
    x_col='Name',
    y_col='HeadCount',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_data,
    directory='/kaggle/working/dataset/preprocessed/train',
    x_col='Name',
    y_col='HeadCount',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)


Found 8186 validated image filenames.
Found 2047 validated image filenames.


In [52]:
def create_model(base_model):
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    return model

# VGG16 model
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_model = create_model(vgg_base)
vgg_model.fit(train_generator, epochs=10, validation_data=val_generator)

# ResNet50 model
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_model = create_model(resnet_base)
resnet_model.fit(train_generator, epochs=10, validation_data=val_generator)


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


256/256 ━━━━━━━━━━━━━━━━━━━━ 149s 475ms/step - loss: 15.9826 - rmse: 3.9875 - val_loss: 9.9747 - val_rmse: 3.1579
Epoch 2/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 106s 403ms/step - loss: 11.4248 - rmse: 3.3778 - val_loss: 8.6463 - val_rmse: 2.9401
Epoch 3/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 108s 410ms/step - loss: 12.5434 - rmse: 3.5346 - val_loss: 7.8493 - val_rmse: 2.8018
Epoch 4/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 106s 402ms/step - loss: 10.1409 - rmse: 3.1830 - val_loss: 7.5533 - val_rmse: 2.7480
Epoch 5/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 106s 402ms/step - loss: 10.2060 - rmse: 3.1926 - val_loss: 7.5420 - val_rmse: 2.7465
Epoch 6/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 106s 401ms/step - loss: 9.4135 - rmse: 3.0669 - val_loss: 6.9363 - val_rmse: 2.6337
Epoch 7/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 106s 403ms/step - loss: 8.6332 - rmse: 2.9379 - val_loss: 12.5294 - val_rmse: 3.5398
Epoch 8/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 106s 403ms/step - loss: 10.2107 - rmse: 3.1898 - val_loss: 10.7891 - val_rmse: 3.2843
Epoch 9/10


In [53]:
# Prepare the test data generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='/kaggle/working/dataset/preprocessed/test',
    x_col='Name',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

# Make predictions with both models
vgg_predictions = vgg_model.predict(test_generator)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'Name': test_df['Name'],
    'HeadCount': vgg_predictions.flatten()
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('/kaggle/working/submission2_vgg_kaggle_main.csv', index=False)

resnet_predictions = resnet_model.predict(test_generator)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'Name': test_df['Name'],
    'HeadCount': resnet_predictions.flatten()
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('/kaggle/working/submission2_resnet_kaggle_main.csv', index=False)

# Average the predictions
final_predictions = (vgg_predictions.flatten() + resnet_predictions.flatten()) / 2

# Round the predictions to the nearest integer
final_head_counts = final_predictions

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'Name': test_df['Name'],
    'HeadCount': final_head_counts
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('/kaggle/working/submission2_kaggle.csv', index=False)


Found 3963 validated image filenames.
124/124 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step
124/124 ━━━━━━━━━━━━━━━━━━━━ 13s 71ms/step
